In [30]:
import pandas as pd
from sentence_transformers import SentenceTransformer

/Users/yangkc/miniforge3/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
questions_df = pd.read_parquet(
    "../data/intermediate/cleaned_arena_data/questions.parquet"
)
threads_df = pd.read_parquet("../data/intermediate/cleaned_arena_data/threads.parquet")

In [3]:
questions_df

,question_id,thread_id,turn_number,user_query,question_role
0,q_00000000,thread_00000000,1,what models are you?,user
1,q_00000001,thread_00000001,1,تجم تفسرلي علاش فما صبع زايد في مرض blackfond ...,user
2,q_00000002,thread_00000002,1,Ellen White über die Flache Erde,user
3,q_00000003,thread_00000003,1,Test,user
4,q_00000004,thread_00000004,1,"Site: ""m.egwwritings.org/en"" über die flache Erde",user
...,...,...,...,...,...
32879,q_00032879,thread_00024066,6,EPDM기반의 식을 알려줘,user
32880,q_00032880,thread_00024067,1,Какой маленький бизнес открыть в новосибирске?,user
32881,q_00032881,thread_00024067,2,А какой можно начать дома с минимальными затра...,user
32882,q_00032882,thread_00024067,3,Сделай анализ по авито,user


In [17]:
questions_df.user_query.nunique()

28907

In [4]:
questions_merged_df = threads_df.merge(
    questions_df[["thread_id", "question_id", "turn_number", "user_query"]],
    on="thread_id",
)

In [22]:
trump_queries = questions_merged_df[
    questions_merged_df.user_query.str.lower().str.contains("trump")
]

In [28]:
trump_queries[["user_query"]].to_csv("./trump_queries.csv", index=None)

In [25]:
for item in trump_queries.user_query:
    print(item)

find me where and when is trump doing speech today after talk with putin
how many government scientists is Trump planning to fire
what time will Trump release the jfk files
donald trump news
Trump deports 238 ‘gang members’ to El Salvador
Why Donald Trump keep saying that tarrifs are paid by the exporter, can you fact check? Could that mean that the importer and the exporter are the same company in many cases?
trump news 3/18/25
dernire controverse de donald trump
Outcomes of March 18 2025 Trump-Putin phone call
Did Trump release the epstein files
why Supreme Court Chief Justice John Roberts rejects Trump's call for impeaching judge who ruled against him
what did trump do yesterday?
what did trump do yesterday?
what did trump do yesterday?
what did trump do yesterday?
which publically listed European companies have seen their share price improve following Trump's recent moves to withdraw / soften US security commitments to Europe / NATO? How much have their SPs risen? 
Trump, Putin cal

In [5]:
questions_merged_df.primary_intent.value_counts()

primary_intent
Factual Lookup         6015
Info Synthesis         5763
Analysis               3568
Explanation            3415
Recommendation         3357
Guidance               3357
Creative Generation    3264
Other                  3044
Text Processing        1100
Name: count, dtype: int64

In [6]:
questions_merged_df.client_country.value_counts()

client_country
US    5134
RU    3197
DE    1862
CN    1401
VN    1141
      ... 
MZ       1
MT       1
MG       1
TG       1
MQ       1
Name: count, Length: 139, dtype: int64

In [7]:
filtered_questions_df = questions_merged_df[
    questions_merged_df.languages.apply(lambda x: len(x) == 1 and "English" in x)
].query("client_country == 'US'")

In [8]:
filtered_questions_df.user_query.value_counts()

user_query
Who are you? Respond with only your name and who trained you.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                

In [9]:
len(questions_merged_df)

32884

In [10]:
questions_merged_df.primary_intent.value_counts()

primary_intent
Factual Lookup         6015
Info Synthesis         5763
Analysis               3568
Explanation            3415
Recommendation         3357
Guidance               3357
Creative Generation    3264
Other                  3044
Text Processing        1100
Name: count, dtype: int64

In [16]:
questions_merged_df.query("primary_intent == 'Info Synthesis'").user_query.sample(10)

20072            Is the low taper fade meme still MASSIVE?
10667    You know the Sonic What If? fancomic about wha...
12322    Which companies have the best remote developme...
11515            Найди мне данные про ПО Сфера и ее модули
22098    Compare the specs between ipad 9th gen and hon...
6287                              Average salary/wage rate
29350                               Latest news on Ukraine
16127                              latest tariff situation
2372               search infamouse graphic design quoztes
13334                What happened to today's stock market
Name: user_query, dtype: object

In [19]:
trump_questions_df = questions_merged_df[
    questions_merged_df.user_query.str.lower().str.contains("trump")
]

In [31]:
model = SentenceTransformer("all-MiniLM-L6-v2")

In [34]:
trump_queries.user_query

8        find me where and when is trump doing speech t...
27       how many government scientists is Trump planni...
78              what time will Trump release the jfk files
209                                      donald trump news
233        Trump deports 238 ‘gang members’ to El Salvador
                               ...                        
28455    Please devise a single panel political cartoon...
28676    Since Trump took office, the U.S. has spent ab...
29674    Elon Musk Out at Tesla? Automaker's Board Repo...
29675    How much is actually news and how much is yell...
30646      what is the latest concensus on trump's tariffs
Name: user_query, Length: 222, dtype: object

In [35]:
embeddings = model.encode(trump_queries.user_query.values)

In [38]:
embeddings

array([[ 0.09154678,  0.00135486,  0.04760856, ..., -0.00419786,
        -0.07835842, -0.03332822],
       [ 0.05847841, -0.00485975,  0.10577406, ..., -0.07445777,
         0.01458153,  0.00947916],
       [ 0.02657595,  0.01695785, -0.01534929, ...,  0.00924313,
        -0.07531891,  0.02373923],
       ...,
       [-0.01999084,  0.01240182,  0.14739536, ..., -0.06165464,
         0.02963084,  0.07473945],
       [ 0.0590217 ,  0.0115913 ,  0.03229765, ..., -0.04271543,
         0.06956494,  0.02363146],
       [-0.02181212, -0.04859022,  0.05283807, ..., -0.03753631,
         0.07481737,  0.07107797]], shape=(222, 384), dtype=float32)